In [24]:
import json
import ijson
import csv
import pandas as pd
import numpy as np

In [5]:
#view the content inside the json file

# Specify the path to your JSON file
json_file_path = 'C:\\Users\\vakee\\veracity_ai\\docs\\dev.jsonl'

# Read the first line of the JSON file
with open(json_file_path, 'r',encoding='utf-8') as file:
    first_line = file.readline()
    # first_line = file.readline()
    # first_line = file.readline()

# Print the first line
print("First Line of JSON File:")
print(first_line.count("short_answers"))

# If you want to parse the JSON content and print it more readable
try:
    json_data = json.loads(first_line)
    print("\nParsed JSON Data:")
    print(json.dumps(json_data, indent=2))
except json.JSONDecodeError as e:
    print(f"\nError decoding JSON: {e}")


First Line of JSON File:
5

Parsed JSON Data:
{
  "annotations": [
    {
      "annotation_id": 13591449469826568799,
      "long_answer": {
        "candidate_index": 92,
        "end_byte": 67824,
        "end_token": 925,
        "start_byte": 66429,
        "start_token": 808
      },
      "short_answers": [
        {
          "end_byte": 66817,
          "end_token": 837,
          "start_byte": 66588,
          "start_token": 816
        }
      ],
      "yes_no_answer": "NONE"
    },
    {
      "annotation_id": 6237931520544082939,
      "long_answer": {
        "candidate_index": 92,
        "end_byte": 67824,
        "end_token": 925,
        "start_byte": 66429,
        "start_token": 808
      },
      "short_answers": [
        {
          "end_byte": 66609,
          "end_token": 819,
          "start_byte": 66588,
          "start_token": 816
        }
      ],
      "yes_no_answer": "NONE"
    },
    {
      "annotation_id": 12127791536449879527,
      "long_answer": 

In [ ]:
#save the sample jason file for processing
output_file_path = 'C:\\Users\\vakee\\veracity_ai\\docs\\sample_file.jsonl'
with open(output_file_path, 'w', encoding='utf-8') as outfile:
    outfile.write(first_line)

In [ ]:
#iterating over the sample file
sample='C:\\Users\\vakee\\veracity_ai\\docs\\sample_file.jsonl'
with open(sample, 'r',encoding='utf-8') as file:
    a=json.loads(file.readline())
    start_token=a['annotations'][1]['short_answers'][0]['start_token']
    end_token=a['annotations'][1]['short_answers'][0]['end_token']
    print(a['document_tokens'][start_token]['token'])
    print(a['document_tokens'][end_token-1]['token'])

In [ ]:

input_file_path = 'C:\\Users\\vakee\\veracity_ai\\docs\\dev.jsonl'
output_file_path = 'C:\\Users\\vakee\\veracity_ai\\docs\\cleaned_file.jsonl'

with open(input_file_path, 'r',encoding='utf-8') as infile, open(output_file_path, 'w', encoding='utf-8') as outfile:
    for line in infile:
        try:
            json.loads(line)
            outfile.write(line)
        except json.JSONDecodeError as e:
            print(f"Ignoring line due to JSON decoding error: {e}")

# Now use the cleaned_file.jsonl with pandas


In [26]:

# Specify the path to your JSONL file and the desired CSV file
jsonl_file_path = 'C:\\Users\\vakee\\veracity_ai\\docs\\cleaned_file.jsonl'  #putting the second backslash for unicode issue
csv_file_path = 'C:\\Users\\vakee\\veracity_ai\\docs\\dev.jsonl2.csv'

# Define the chunk size (adjust as needed)
chunk_size = 1000

# Create an iterator to read the JSONL file in chunks
jsonl_reader = pd.read_json(jsonl_file_path, lines=True, chunksize=chunk_size)
# Iterate through chunks and append to CSv
for i, chunks in enumerate(jsonl_reader):
    #print(chunk)
    # we need`document_title`, `question_text`, `short_answer`, `yes_no_answer` columns.
    # Create a DataFrame with only the 'short_answer and yes_no_answer' column
    # Create a DataFrame with  short answer 1 and so on
    # Retrieve the 'short_answers' key from the 'annotations' column
    short_answers_list = []
    yes_no_answer_list = []
    for j,chunk in enumerate(chunks["annotations"]): #runs chunk_size times
        short_answers=[]
        yes_no_answer=[]
        for k,sub_chunk in enumerate(chunk): #runs for 5 times
            if (sub_chunk['short_answers']!=[] ):
                start_token=sub_chunk['short_answers'][0]['start_token']
                end_token=sub_chunk['short_answers'][0]['end_token']
                try:
                    ans=chunks['document_tokens'][j][start_token]['token'] +" "+ chunks['document_tokens'][j][end_token-1]['token']
                except:
                    ans=None
            else:
                ans=None
            #print(ans)
            short_answers.append(ans)
            yes_no_answer.append(sub_chunk.get('yes_no_answer',None))
        # short_answers = [item.get('short_answers', None) for item in annotations]
        short_answers_list.append(short_answers)
        yes_no_answer=[i for i in yes_no_answer if i != 'NONE']
        if yes_no_answer==[]:
            yes_no_answer_list.append(None)
        else:
            yes_no_answer_list.append(list(set(yes_no_answer))[0])
        
    # short_answers_list = [item.get('short_answers', None) if isinstance(item, dict) else None for item in chunk['annotations']]

    # Create a DataFrame with 'short_answers_0', 'short_answers_1', etc. columns
    short_answers_df = pd.DataFrame(short_answers_list)#.add_prefix('short_answers')   #here bro
    short_answers_df.columns = [f'short_answer{int(c)+1}' for c in short_answers_df]
    # yes_no_answer = [item.get('short_answers', None) if isinstance(item, dict) else None for item in chunk['annotations']]
    yes_no_answer_df = pd.DataFrame({'yes_no_answer': yes_no_answer_list})
    # Concatenate the original chunk with the required data
    chunk_concatenated = pd.concat([chunks['document_title'], chunks['question_text'] ,short_answers_df, yes_no_answer_df], axis=1) #
    # We drop any rows with NaN values
    chunk_concatenated.dropna(axis = 0)

    if i == 0:
        print("i == 0")
        # For the first chunk, create the CSV file with header
        chunk_concatenated.to_csv(csv_file_path, index=False, mode='w')
    else:
        print("i >0")
        # For subsequent chunks, append to the existing CSV file without header
        chunk_concatenated.to_csv(csv_file_path, index=False, header=False, mode='a')
    print(f'Processed chunk {i + 1}')
print('Conversion completed.')


i == 0
Processed chunk 1
i >0
Processed chunk 2
i >0
Processed chunk 3
i >0
Processed chunk 4
i >0
Processed chunk 5
i >0
Processed chunk 6
Conversion completed.


In [18]:
# Specify the path to your CSV file
csv_file_path = 'C:\\Users\\vakee\\veracity_ai\\docs\\dev.jsonl2.csv'

# Use pd.read_csv to read the first few rows (adjust nrows as needed)
df_sample = pd.read_csv(csv_file_path, nrows=10)

df_sample

,document_title,question_text,short_answer1,short_answer2,short_answer3,short_answer4,short_answer5,yes_no_answer
0,Therefore sign,what do the 3 dots mean in math,the syllogism,the sign,NaN,a syllogism,therefore sign,NaN
1,Watchman (law enforcement),when was the writ watch invented by who,NaN,NaN,NaN,NaN,NaN,NaN
2,Photograph (Ringo Starr song),who wrote the song photograph by ringo starr,NaN,Ringo Starr,Richard Starkey,NaN,Ringo Starr,NaN
3,Super Bowl 50 halftime show,who is playing the halftime show at super bowl...,Coldplay Mars,Coldplay Coldplay,NaN,British Mars,Coldplay Coldplay,NaN
4,Matt Lanter,star wars the clone wars anakin voice actor,NaN,Matthew Lanter,Matthew Lanter,Matthew Lanter,NaN,NaN
5,A Whiter Shade of Pale,meaning behind the song whiter shade of pale,NaN,NaN,NaN,NaN,NaN,NaN
6,Globe,where is most distortion found on a globe,NaN,NaN,NaN,NaN,NaN,NaN
7,List of A Certain Scientific Railgun episodes,to aru kagaku no railgun s episode 3,NaN,NaN,NaN,NaN,Project Noise,NaN
8,List of Seraph of the End chapters,when do new chapters of owari no seraph come out,NaN,NaN,NaN,NaN,NaN,NaN
9,Surname,where does the last name painter come from,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#filter by number
age_selection = (30,40)
mask = df['age'].between(*age_selection)
df[mask]    

#filter by department
departments = df['Department'].unique().tolist()
department_selection = ['Sales', 'Marketing']
mask = df['Department'].isin(department_selection)
df[mask]

#group by rating
df_grouped = df.groupby(by=['Rating']).count()[['Age']]
df_grouped
bar_chart = px.bar(df_grouped,
                    x=df_grouped.index, 
                    y='Age', 
                    text='Votes',
                    color_discrete_sequence=['#D63230']*len(df_grouped),
                    template= 'plotly_dark',)
bar_chart.show()